### Imports

In [1]:
import pandas as pd

### Read data from 'xlsx' file

In [2]:
# Read 'xlsx' file as dataframe
df = pd.read_excel('../data/processed/data_cuba.xlsx', index_col=None, sheet_name='base_limpia')
# Create a list with dataframe column names
cols = df.columns

0- ID de respuesta
1- Fecha de envío
2- Última página
3- Fecha de inicio
4- Fecha de la última acción
5- Código único respondiente
6- Datos personales [Ciudad de residencia actual]
7- Datos personales [Fecha de nacimiento]
8- Datospersonales[Fechanacimiento] limpia
9- Año de nacimiento
10- Datos personales [Identidad racial]
11- Datos personales [Total de partos/cesáreas]
12- Datos de contacto [Correo electónico]
13- ¿En el orden de todos tus partos, qué lugar ocupa este?
14- ¿Fuiste parto múltiple?
15- En caso positivo, indícanos cuántos bebés fueron [Mellizos]
16- En caso positivo, indícanos cuántos bebés fueron [Trillizos]
17- En caso positivo, indícanos cuántos bebés fueron [Cuatrillizos]
18- En caso positivo, indícanos cuántos bebés fueron [Otro]
19- En caso de señalar otro, por favor especifica
20- ¿Cuál fue el sexo de tu bebé al nacer?  [Femenino]
21- ¿Cuál fue el sexo de tu bebé al nacer?  [Masculino]
22- ¿Cuánto pesó al nacer aproximadamente?  [Libras:][]
23- ¿Cuánto pesó al n

### Create pivot tables from dataframe

In [13]:
"""
Function to create 
pivot tables from
columns 145-155
"""
def create_pivot_table():
    dfs = []
    count = 145
    for i in range(11):
        # Create a pivot table from df
        table = pd.pivot_table(df, values=cols[0], index=cols[27],
                                columns=cols[count], aggfunc='count')
        count += 1
        # Convert index to columns and reset index
        table = table.rename_axis('Hospital').reset_index().rename_axis('', axis='columns')
        # Drop a column by index
        table.drop(table.columns[-1], axis=1, inplace=True)
        table[f'Consecuencia{i+1}'] = table.sum(axis=1)
        # Drop rows with a certain value
        table.drop(table.loc[table[f'Consecuencia{i+1}'] == 0].index, inplace=True)
        # Create a dataframe with selected columns
        table = table[['Hospital', f'Consecuencia{i+1}']]
        # Reset index
        table= table.reset_index(drop=True)
        # Append table to list
        dfs.append(table)
    return dfs

dfs = create_pivot_table()

c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11 = dfs[0], dfs[1], dfs[2], dfs[3], dfs[4], dfs[5], dfs[6], dfs[7], dfs[8], dfs[9], dfs[10]

/tmp/ipykernel_20571/1520375655.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  table[f'Consecuencia{i+1}'] = table.sum(axis=1)


### Clean data

In [14]:
"""
Function to remove 
whitespace and new line
at the beginning
"""
def clean_dataframes(*args, **kwargs):
    dfs = list(args)
    for i in range(len(dfs)):
        dfs[i] = dfs[i]['Hospital'].replace(r'^\s|\n', '', regex=True, inplace=True)
    return dfs

clean_dataframes(c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11)

[None, None, None, None, None, None, None, None, None, None, None]

### Merge dataframes

In [15]:
"""
Function to merge dataframes
"""
def mergeDataframes(*args, **kwargs):
    dfs = list(args)
    for i in range(len(dfs)):
        if i <= 1:
            df = pd.merge(dfs[0], dfs[1], on='Hospital', how ='outer')
        else:
            df = pd.merge(df, dfs[i], on='Hospital', how ='outer')
    return df

consec = mergeDataframes(c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11)

# Regular expressions to clean data typos
consec['Hospital'].replace(r'^\s', '', regex=True, inplace=True)

consec.shape

(41, 12)

In [16]:
# Read 'xlsx' file as dataframe
directorio = pd.read_excel('../data/processed/directorio_hospitales.xlsx', index_col=None, sheet_name='base_limpia', converters={'Long':str, 'Lat': str})

# Merge dataframes by a column
df_join = pd.merge(consec, directorio, on='Hospital', how='left')

# Remove rows with NaN values on column
df_join.dropna(subset=['Long'], inplace=True)
# Reset index
df_join = df_join.reset_index(drop=True)

# Save dataframe as 'xlsx' file
# df_join.to_excel('../data/processed/secuelas_hospitales.xlsx', index = False)
# df_join.to_json('../data/processed/json/secuelas_hospitales.json', orient='index')

df_join.sample(3)

,Hospital,Consecuencia1,Consecuencia2,Consecuencia3,Consecuencia4,Consecuencia5,Consecuencia6,Consecuencia7,Consecuencia8,Consecuencia9,Consecuencia10,Consecuencia11,Long,Lat
15,Hospital Provincial Ginecobstétrico Julio Rafa...,2.0,1.0,NaN,2.0,3.0,2.0,1.0,4.0,2.0,4.0,3.0,23.05381783218849,-81.5667992219682
34,Hospital General Docente Héroes de Baire,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,2.0,2.0,2.0,21.8913073212293,-82.80654867
12,Hospital Ginecobstétrico América Arias (Matern...,2.0,4.0,1.0,7.0,11.0,4.0,3.0,11.0,14.0,12.0,12.0,23.14226057538131,-82.39222113
